1. 이미지 사이즈 전처리 ㅇ
    - 전처리된 이미지 불러와서 -> 텐서저장-> 1줄짜리로

2. 딥러닝 모델 클래스 적기 (nn.Module 상속)
3. 딥러닝 모델 객체 만들어서 학습시키기 (학습, 테스트 데이터 분리)
4. 학습된 객체에 테스트 데이터 넣어서 예측값 뽑기
5. 뽑아낸 예측값 데이터 사용해서 성능평가하기
6. 이렇게 여러 클래스 모델 만들어서 학습, 평가 반복

In [2]:
import pandas as pd
import numpy as np
from PIL import Image
import os,sys
import torch
import torchviz
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

file = './Dog Emotion/labels.csv'

[1] 데이터 전처리

In [3]:
# 데이터 불러오기
df = pd.read_csv(file, usecols=[1, 2])

In [4]:
#df.label.unique()
#filename = df.filename.to_list() 

In [5]:
#각 감정별로 파일 분리
angry = df[df['label']=='angry']
angryfile = angry.filename.to_list()

sad = df[df['label']=='sad']
sadfile = sad.filename.to_list()

relaxed = df[df['label']=='relaxed']
relaxedfile = relaxed.filename.to_list()

happy = df[df['label']=='happy']
happyfile = happy.filename.to_list()

In [6]:
pathList = [[],[],[],[]]
for i in angry.filename :
    img_path = os.path.join('./Dog Emotion/angry/'+i)
    pathList[0].append(img_path)

for i in sad.filename :
    img_path = os.path.join('./Dog Emotion/sad/'+i)
    pathList[1].append(img_path)

for i in relaxed.filename :
    img_path = os.path.join('./Dog Emotion/relaxed/'+i)
    pathList[2].append(img_path)

for i in happy.filename :
    img_path = os.path.join('./Dog Emotion/happy/'+i)
    pathList[3].append(img_path)

In [7]:
Image.ANTIALIAS = Image.LANCZOS
size = 128, 128 # 바꾸고 싶은 사이즈
sad_path = './Dog Emotion/sad/'
angry_path = './Dog Emotion/angry/'
happy_path = './Dog Emotion/happy/'
relaxed_path = './Dog Emotion/relaxed/'
modified_path = 'C:/Users/KDP-48/PythonProject/Data/KDT-09-DL/miniproject/Dog Emotion/image/'
modified_path2 = 'C:/Users/KDP-48/PythonProject/Data/KDT-09-DL/miniproject/Dog Emotion/happy2/'

In [8]:
print(os.getcwd()) # 현재 작업하고 있는 디렉토리 출력

c:\Users\KDP-48\PythonProject\Data\KDT-09-DL\miniproject


In [9]:
type(os.listdir(sad_path)[0]) #지정된 경로에 있는 파일과 디렉토리의 목록을 반환하는 함수

str

In [10]:
def resize_and_crop(img_path, modified_path, size, crop_type='middle'):
    files = os.listdir(img_path)

    for file in files:
        img = Image.open(img_path+file)
        img_ratio = img.size[0]/float(img.size[1])
        ratio = size[0] / float(size[1])

        if ratio>img_ratio :
            img = img.resize((size[0], int(round(size[0]*img.size[1]/img.size[0]))),
                             Image.ANTIALIAS)
            if crop_type == 'middle':
                box = (0, int(round((img.size[1] - size[1]) / 2)), 
                       img.size[0], int(round((img.size[1] + size[1]) / 2)))
            img = img.crop(box)
        
        elif ratio < img_ratio:
            img = img.resize((int(round(size[1] * img.size[0] / img.size[1])), size[1]),
                             Image.ANTIALIAS)
            if crop_type == 'middle':
                box = (int(round((img.size[0] - size[0]) / 2)), 0, 
                       int(round((img.size[0] + size[0]) / 2)), img.size[1])
            img = img.crop(box)
        else:
            img = img.resize((size[0], size[1]), Image.ANTIALIAS)

        img.save(modified_path + file)

In [11]:
# resize_and_crop(sad_path, modified_path, size)
# resize_and_crop(angry_path, modified_path, size)
# resize_and_crop(happy_path, modified_path2, size)
# resize_and_crop(relaxed_path, modified_path, size)

In [12]:
file_path = './Dog Emotion/image/'
imgList=[]
for pth in os.listdir(file_path) :
    imgList.append(file_path+pth)
print(imgList)

['./Dog Emotion/image/001ndHD2FL3RHNrUp29Qp4WvTiHo5K388.jpg', './Dog Emotion/image/02moEqkRoDqAbzoLSBOuzFd3iVK53n423.jpg', './Dog Emotion/image/02rDGGot7kGU3cKIB5qhOfPaetpRNq548.jpg', './Dog Emotion/image/04f9aAUe5glmLufixJGSc9Yv1X2mRU123.jpg', './Dog Emotion/image/04SQoOw0cFD5lsACrLRCmUWFPu16AU746.jpg', './Dog Emotion/image/09dUVMcjCDfOtbeYDQg5Fvu3GPHWJg811.jpg', './Dog Emotion/image/09qhORmLRHfy6HgNlXTDlqJVk1NJod130.jpg', './Dog Emotion/image/0A7QpoL9r4k80HkzQBLCi7biz0K7fo580.jpg', './Dog Emotion/image/0aNyXBrmNA7XdefwHvgO2n1rnpqQAp885.jpg', './Dog Emotion/image/0AvKtuzA7LfxnKaO0bey9mQMLnxXad73.jpg', './Dog Emotion/image/0biYFWF4fFWJOTHpD1ONSknqtHvLsS70.jpg', './Dog Emotion/image/0BntC5Yeh254FCqZO9f7Rd88myxmm5172.jpg', './Dog Emotion/image/0C5yo7GxMy8lztxNZvSdfEx2gSPRTR701.jpg', './Dog Emotion/image/0c6mu9ObWcjUySz1yUu6XgTYyA4jO3739.jpg', './Dog Emotion/image/0C9jEgFQHsh36W5U2u5CA98lB7C5eX806.jpg', './Dog Emotion/image/0CD90u75I6WgftgwIvq0pNM6rsHInC795.jpg', './Dog Emotion/image/0cjz

In [13]:
gsImgList =[] # 그레이 스케일로 변경한 그림파일np를 담을 리스트.
for i in imgList :
    img = Image.open(i) #그림파일 열기
    img = img.convert('L') #그레이 스케일로 변경
    img = np.array(img)
    #print(img.shape)
    gsImgList.append(abs(img.astype('int')-255)) # img를 int로 변경. 흑백 반전을 위해 - 255 한 후 절댓값 변환.
#gsImgList

In [14]:
#print(gsImgList[0].shape, len(gsImgList[0]))

# 128*128 크기의 그림이므로 열의 개수는 16384

In [15]:
# 모든 이미지 데이터를 1차원 데이터로 펼쳐서 total_list에 저장.
total_list = [] 
for i in range(1000):
    sample_list = []
    for row in range(len(gsImgList[0])):
        
        for k in range(len(gsImgList[row])):
            sample_list.append(gsImgList[i][row][k])
    total_list.append(sample_list)
print(len(total_list))

1000


In [16]:
file_path2 = './Dog Emotion/happy2/'
imgList2=[]
for pth2 in os.listdir(file_path2) :
    imgList2.append(file_path2+pth2)
#print(imgList2)
    
gsImgList2 =[]
for i in imgList2 :
    img2 = Image.open(i) #그림파일 열기
    img2 = img2.convert('L') #그레이 스케일로 변경
    img2 = np.array(img2)
    #print(img.shape)
    gsImgList2.append(abs(img2.astype('int')-255)) # img를 int로 변경. 흑백 반전을 위해 - 255 한 후 절댓값 변환.
#gsImgList2

In [17]:
total_list2 = [] 
for i in range(1000):
    sample_list2 = []
    for row in range(len(gsImgList2[0])):
        
        for k in range(len(gsImgList2[row])):
            sample_list2.append(gsImgList2[i][row][k])
    total_list2.append(sample_list2)
print(len(total_list2))

1000


In [18]:
print(len(total_list))
print(len(total_list2))

1000
1000


In [19]:
other = pd.DataFrame(total_list)
happy = pd.DataFrame(total_list2)

In [27]:
# 행복한 강아지 사진 식별을 위해 컬럼명 추가.
happy['happy']=1

In [28]:
# 행복한 강아지 사진 식별을 위해 컬럼명 추가
other['happy']=0

In [29]:
happydog = pd.concat([happy, other], axis=0)
happydog

,0,1,2,3,4,5,6,7,8,9,...,16375,16376,16377,16378,16379,16380,16381,16382,16383,happy
0,199,196,196,199,207,212,214,214,195,177,...,249,173,241,245,255,201,134,104,50,1
1,186,185,202,206,193,198,198,174,152,157,...,188,191,206,230,229,216,223,218,184,1
2,241,237,233,235,240,243,242,239,239,238,...,241,228,234,227,232,240,231,234,240,1
3,158,164,148,146,162,148,136,162,145,149,...,131,159,132,135,112,105,78,103,115,1
4,166,171,176,179,183,187,188,186,183,182,...,145,142,149,143,117,132,134,149,122,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,22,23,24,25,26,26,26,26,26,26,...,116,110,105,96,87,82,85,94,102,0
996,146,151,139,135,141,103,116,88,112,119,...,119,119,86,100,121,104,105,116,98,0
997,93,91,74,70,88,82,70,77,93,95,...,141,145,169,170,170,196,208,199,196,0
998,93,79,99,102,101,101,91,116,102,94,...,137,152,151,148,145,144,151,162,171,0


In [30]:
happydog.to_csv('happydog.csv')